In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [2]:
sf_data = pd.read_csv("./raw_data/sanfrancisco.csv")

In [37]:
raw_columns = list(sf_data)

In [4]:
unused_columns = ['snow_1h', 'snow_24h', 'rain_24h', 'rain_1h', 'snow_3h', 'rain_today', 'snow_today', 'weather_icon', 'weather_id', 'sea_level', 'grnd_level', 'lat', 'lon', 'city_id', 'city_name']

In [16]:
used_columns = columns = list(set(raw_columns) - set(unused_columns))

In [53]:
sf_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38415 entries, 0 to 38414
Data columns (total 28 columns):
dt                     38415 non-null int64
dt_iso                 38415 non-null object
city_id                38415 non-null int64
city_name              0 non-null float64
lat                    0 non-null float64
lon                    0 non-null float64
temp                   38415 non-null int64
temp_min               38415 non-null float64
temp_max               38415 non-null float64
pressure               38415 non-null int64
sea_level              0 non-null float64
grnd_level             0 non-null float64
humidity               38415 non-null int64
wind_speed             38415 non-null int64
wind_deg               38415 non-null int64
rain_1h                1866 non-null float64
rain_3h                1355 non-null float64
rain_24h               97 non-null float64
rain_today             137 non-null float64
snow_1h                3 non-null float64
snow_3h          

In [58]:
def cleanup(raw_data, used_columns):
    data = raw_data.loc[:, used_columns] 
    print("fill_na")
    data['rain_3h'] = data['rain_3h'].fillna(0)
    print("drop_duplicates")
    data.drop_duplicates('dt', inplace=True)
    print("add_new_dada")
    data = add_new_data(data)
    
    unused_columns  = ['dt_iso', 'weather_main', 'weather_description']
    
    data = data.drop(unused_columns, axis=1)
    data = data.reset_index(drop=True)
    return data

def add_new_data(data):
    data['dt_datetime'] =  pd.to_datetime(data['dt_iso'], format='%Y-%m-%d %H:%M:%S +%f %Z')
    weather_description_columns = list(set(data['weather_description']))
    weather_main_columns = list(set(data['weather_main']))
    data = transform_categorical_data(data, weather_description_columns,weather_main_columns)

    return data

def transform_categorical_data(data, weather_description_columns, weather_main_columns):
    for column in weather_description_columns:
        data[column] = data['weather_description'] == column
        data[column] = data[column].astype(int)
        
    for column in weather_main_columns:
        data[column] = data['weather_main'] == column
        data[column] = data[column].astype(int)
    return data
    

In [59]:
sf_data2 = cleanup(sf_data, used_columns)


fill_na
drop_duplicates
add_new_dada


In [60]:
sf_data2.head(10)

,temp_min,dt,rain_3h,wind_deg,temp_max,clouds_all,pressure,humidity,temp,wind_speed,...,Rain,Mist,Clear,Drizzle,Clouds,Snow,Smoke,Fog,Squall,Haze
0,282.15,1349096400,0.0,150,301.15,0,1009,88,0,2,...,0,0,0,1,0,0,0,0,0,0
1,282.15,1349186400,0.0,0,301.15,1,1015,63,0,0,...,0,0,1,0,0,0,0,0,0,0
2,283.15,1349190000,0.0,0,302.15,1,1016,51,0,0,...,0,0,1,0,0,0,0,0,0,0
3,287.15,1349193600,0.0,190,307.15,0,1011,56,0,4,...,0,0,0,0,0,0,0,0,0,1
4,290.15,1349197200,0.0,140,307.15,0,1001,94,0,3,...,0,0,0,0,0,0,0,0,0,1
5,293.71,1349200800,0.0,230,305.93,40,1009,70,0,4,...,0,0,0,0,1,0,0,0,0,0
6,293.15,1349204400,0.0,170,308.15,90,1000,83,0,6,...,0,0,0,0,1,0,0,0,0,0
7,293.15,1349208000,0.0,250,308.71,0,1007,75,0,3,...,0,0,0,0,0,0,0,0,0,1
8,295.15,1349211600,0.0,260,308.71,75,1007,75,0,3,...,0,0,0,0,1,0,0,0,0,0
9,296.15,1349215200,0.0,300,309.26,1,1013,18,0,6,...,0,0,1,0,0,0,0,0,0,0


In [67]:
def add_previous_datum(raw_data):
    data = raw_data.copy()
    diff_columns = list(set(raw_data.columns) - set(['dt', 'dt_iso', 'dt_datetime']))
    maximum_prev = 2 *  24
    data = data.apply(add_previous_data, args=(diff_columns, data, maximum_prev), axis=1)
    return data

def add_previous_data(current_data,  diff_columns, raw_data, maximum_prev):
    index = current_data.name
    print(index)
    if index == 0:
        return current_data
    prev_right = index - 1
    prev_left = index - maximum_prev if index - maximum_prev > 0 else 0
       
    while prev_left <= prev_right:
        current_data = add_diff_data(current_data, raw_data.iloc[prev_left], maximum_prev, diff_columns)
        prev_left += 1
    return current_data

def add_diff_data(current_data, prev_data, maximum_prev, diff_columns):
    diff = int(pd.Timedelta(current_data['dt_datetime'] - prev_data['dt_datetime']).seconds/ 3600)
    if diff > 0 and diff < maximum_prev:
        for diff_column in diff_columns:
            column_name = '{}_{}_ago'.format(diff_column, diff)
            current_data[column_name] = prev_data[diff_column]
    return current_data

# add new data by merging np array and adding dummy data 
[NALL, NALL, data1, data2]


NameError: name 'month' is not defined

In [77]:
def add_datetime(raw_data):
    data = raw_data.copy()
    data = data.apply(transform_datetime)
    data = data.drop(['dt_datetime'], axis=1)
    return data
    
def transform_datetime(current_data):
    for month in range(1, 12):
        current_data['month_{}'.format(month)] = 1 if current_data['dt_datetime'].month == month else 0
       
    current_data['year'] =  current_data['dt_datetime'].year
    current_data['dayofweek'] = current_data['dt_datetime'].dayofweek
    current_data['dayofyear'] = current_data['dt_datetime'].dayofyear
    current_data['hourofday'] = current_data['dt_datetime'].hour
    return current_data

In [68]:
def add_target_datum(raw_data):
    data = raw_data.copy()
    data = data.apply(args=(data))
    return data

def add_target_data(current_data, raw_data):
    index = current_data.name
    last_index = raw_data.index[-1]
    if index == last_index:
        return current_data
    
    current_data['target_temperature'] = raw_data.iloc[index+1]['temparature']
    return current_data

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [73]:
sf_data2.index[-1]

0